In [275]:
import pandas as pd
import statsmodels.api as sm
import re
import numpy as np
import pandas as pd
from itertools import product

In [276]:
def remove_pattern(col_name):
    return re.sub(r'_[0-9]{3}$', '', col_name)

Zones = pd.read_excel('Zones.xlsx')
Dataset = pd.read_csv('KAG 2016-17\Agriculture\Agriculture_KAG_2016_17.csv')
Dataset.columns = map(remove_pattern, Dataset.columns)

param = pd.read_csv('Params/3_parameters_multicollinearity_rice.csv')
param.columns = map(remove_pattern, param.columns)

Working_Directory = pd.read_csv('Working_Directory.csv')
Working_Directory = Working_Directory[['District','Rice_Production', 'TotalNPK','AreaUnderCereals_Paddy','NetAreaIrrigated_Canals_NetAreaIrrigated','Rice_Ratio']].copy()

for x in param.columns:
    if(x != 'AreaUnderCereals_Paddy'):
        Working_Directory[x] = Working_Directory[x]*Working_Directory['Rice_Ratio']

Working_Directory

,District,Rice_Production,TotalNPK,AreaUnderCereals_Paddy,NetAreaIrrigated_Canals_NetAreaIrrigated,Rice_Ratio
0,BENGALURU,3376.0,513.822990,782.0,0.000000,0.022043
1,BENGALURU(R),2354.0,167.835355,736.0,0.000000,0.008715
2,RAMANAGARA,12128.0,257.908893,4468.0,272.509463,0.034517
3,CHITRADURGA,4566.0,183.032957,1297.0,9.283245,0.005101
4,DAVANAGERE,489505.0,21197.529554,98259.0,18037.386429,0.212733
5,KOLAR,864.0,108.445953,395.0,0.000000,0.004887
6,CHIKKABALLAPURA,2698.0,209.640893,1333.0,0.000000,0.007393
7,SHIVAMOGGA,384974.0,17722.015760,109617.0,13305.677787,0.313060
8,TUMAKURU,26042.0,1069.562417,10379.0,114.924409,0.028037
9,CHIKKAMAGALURU,110533.0,14694.297645,33036.0,1344.644186,0.216529


In [277]:
Case_file = pd.read_csv('Params/3_parameters_multicollinearity_rice.csv')
y = Working_Directory['Rice_Production']
Case_file[:5]

,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated
0,782.0,513.822990,0.000000
1,736.0,167.835355,0.000000
2,4468.0,257.908893,272.509463
3,1297.0,183.032957,9.283245
4,98259.0,21197.529554,18037.386429


In [278]:
if(len(Case_file) < 31):
    row_24 = Case_file.iloc[24]
    halved_row_24 = row_24 / 2
    Case_file.iloc[24] = halved_row_24
    Case_file.loc[len(Case_file)] = halved_row_24

Case_file[24:]

,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated
24,46832.5,16648.820502,20953.141802
25,1548.0,181.920490,0.000000
26,4199.0,383.098388,148.020607
27,49358.0,30981.381245,30897.307860
28,115246.0,41912.073068,39866.073203
29,50753.0,14075.849458,7221.642581
30,46832.5,16648.820502,20953.141802


In [279]:
# Ensure that the index of y and Case_file are aligned
y = y.loc[Case_file.index]

# Identify rows in Case_file that have any zeros
rows_to_remove = (Case_file == 0).any(axis=1)

# Apply the mask to both DataFrames
# Case_file = Case_file[~rows_to_remove].reset_index(drop=True)
# y = y[~rows_to_remove].reset_index(drop=True)

mean_value = Working_Directory[~rows_to_remove]['NetAreaIrrigated_Canals_NetAreaIrrigated'].mean()
Working_Directory.loc[rows_to_remove, 'NetAreaIrrigated_Canals_NetAreaIrrigated'] = mean_value
Case_file['NetAreaIrrigated_Canals_NetAreaIrrigated'] = Working_Directory['NetAreaIrrigated_Canals_NetAreaIrrigated']*Working_Directory['Rice_Ratio']


# Identify rows where 'Rice_Production' is less than 500
rows_to_remove = Working_Directory['Rice_Production'] < 500

# Apply the mask to both DataFrames
Case_file = Case_file[~rows_to_remove].reset_index(drop=True)
y = y[~rows_to_remove].reset_index(drop=True)

# Correct way to get the length of Case_file
case_file_length = len(Case_file)
case_file_length

29

In [280]:
Case_file.columns

Index(['AreaUnderCereals_Paddy', 'TotalNPK',
       'NetAreaIrrigated_Canals_NetAreaIrrigated'],
      dtype='object')

In [281]:
Case_file =sm.add_constant(Case_file)
Case_file.head()

,const,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated
0,1.0,782.0,513.822990,211.347216
1,1.0,736.0,167.835355,83.555490
2,1.0,4468.0,257.908893,9.406131
3,1.0,1297.0,183.032957,0.047351
4,1.0,98259.0,21197.529554,3837.140539


In [282]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

columns_to_standardize = ['AreaUnderCereals_Paddy', 'TotalNPK', 'NetAreaIrrigated_Canals_NetAreaIrrigated']

Case_file_1 = Case_file[['const'] + columns_to_standardize].copy()
scaler = MinMaxScaler()

X_st = pd.DataFrame()
X_st['const'] = Case_file['const']
X_st[columns_to_standardize] = scaler.fit_transform(Case_file_1[columns_to_standardize])

Target = 0.05
y_st = pd.DataFrame()
y_st['Rice_Production'] = Working_Directory[~rows_to_remove]['Rice_Production'].reset_index(drop=True)
columns_to_standardize = ['Rice_Production']
y_st[columns_to_standardize] = scaler.fit_transform(y_st[columns_to_standardize])
# y_st['Target_Production'] = (1+Target)*y_st['Maize_Production']
# columns_to_standardize = ['GroundNut Production','Target Production']
y_st.head()

,Rice_Production
0,0.005111
1,0.003032
2,0.022919
3,0.007532
4,0.994228


In [283]:
# model=sm.OLS(y,Case_file).fit()
# print(model.summary())
# model.params.values
# coef=model.params.values[1:]
# coef

model=sm.OLS(y_st['Rice_Production'],X_st).fit()
Case_file = X_st
print(model.summary())
model.params.values
coef=model.params.values[1:]
coef

                            OLS Regression Results                            
Dep. Variable:        Rice_Production   R-squared:                       0.944
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     140.2
Date:                Thu, 19 Sep 2024   Prob (F-statistic):           9.23e-16
Time:                        16:34:28   Log-Likelihood:                 36.408
No. Observations:                  29   AIC:                            -64.82
Df Residuals:                      25   BIC:                            -59.35
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

array([ 0.75076178,  0.28794102, -0.00368225])

In [284]:
Case_file.drop('const',axis=1,inplace=True)
Case_file[:5]

,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated
0,0.003370,0.009697,0.014826
1,0.002969,0.001421,0.005859
2,0.035463,0.003575,0.000657
3,0.007854,0.001784,0.000000
4,0.852095,0.504480,0.269227


In [285]:
Case_file.loc[:,'Beta_0'] = model.params.values[0]
Case_file.loc[:,'Beta_1'] = coef[0]
Case_file.loc[:,'Beta_2'] = coef[1]
Case_file.loc[:,'Beta_3'] = coef[2]

intervals = np.arange(0, 1.1, 0.1)
columns=['Sensitivity_AreaUnderCereals_Paddy', 'Sensitivity_TotalNPK','Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated']
# Generate all possible combinations of elements in arr
num = len(columns)
combinations = product(intervals, repeat=len(columns))
combinations

In [286]:
alpha_df = pd.DataFrame(combinations, columns=columns)
alpha_df

,Sensitivity_AreaUnderCereals_Paddy,Sensitivity_TotalNPK,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated
0,0.0,0.0,0.0
1,0.0,0.0,0.1
2,0.0,0.0,0.2
3,0.0,0.0,0.3
4,0.0,0.0,0.4
...,...,...,...
1326,1.0,1.0,0.6
1327,1.0,1.0,0.7
1328,1.0,1.0,0.8
1329,1.0,1.0,0.9


In [287]:
Case_file['key'] = 1
alpha_df['key'] = 1

# Performing the Cartesian product
result_df = pd.merge( alpha_df, Case_file, on='key').drop('key', axis=1)

print(result_df)

       Sensitivity_AreaUnderCereals_Paddy  Sensitivity_TotalNPK  \
0                                     0.0                   0.0   
1                                     0.0                   0.0   
2                                     0.0                   0.0   
3                                     0.0                   0.0   
4                                     0.0                   0.0   
...                                   ...                   ...   
38594                                 1.0                   1.0   
38595                                 1.0                   1.0   
38596                                 1.0                   1.0   
38597                                 1.0                   1.0   
38598                                 1.0                   1.0   

       Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated  \
0                                                    0.0      
1                                                    0.0      
2     

In [288]:
result_df.head()

,Sensitivity_AreaUnderCereals_Paddy,Sensitivity_TotalNPK,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated,Beta_0,Beta_1,Beta_2,Beta_3
0,0.0,0.0,0.0,0.003370,0.009697,0.014826,-0.029489,0.750762,0.287941,-0.003682
1,0.0,0.0,0.0,0.002969,0.001421,0.005859,-0.029489,0.750762,0.287941,-0.003682
2,0.0,0.0,0.0,0.035463,0.003575,0.000657,-0.029489,0.750762,0.287941,-0.003682
3,0.0,0.0,0.0,0.007854,0.001784,0.000000,-0.029489,0.750762,0.287941,-0.003682
4,0.0,0.0,0.0,0.852095,0.504480,0.269227,-0.029489,0.750762,0.287941,-0.003682


In [289]:
Working_Directory['Rice_Production'].mean()

131114.06451612903

In [290]:
delta_O = 0.05*y_st['Rice_Production'].median()
delta_O

0.012671777780490685

In [291]:
# delta_O = 5000

result_df['New_F1'] = result_df['AreaUnderCereals_Paddy'] + (result_df['Sensitivity_AreaUnderCereals_Paddy']*delta_O)/result_df['Beta_1']
result_df['New_F2'] = result_df['TotalNPK'] + (result_df['Sensitivity_TotalNPK']*delta_O)/result_df['Beta_2']
result_df['New_F3'] = result_df['NetAreaIrrigated_Canals_NetAreaIrrigated'] + (result_df['Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated']*delta_O)/result_df['Beta_3']
# result_df['District'] = np.tile(Working_Directory[~rows_to_remove]['District'],11**num)
result_df['District'] = np.tile(Working_Directory[~rows_to_remove]['District'],11**num)

# result_df['New_F1'] = result_df['New_F1'].apply(lambda x: max(0, x))
# result_df['New_F2'] = result_df['New_F2'].apply(lambda x: max(0, x))
# result_df['New_F3'] = result_df['New_F3'].apply(lambda x: max(0, x))

result_df

,Sensitivity_AreaUnderCereals_Paddy,Sensitivity_TotalNPK,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District
0,0.0,0.0,0.0,0.003370,0.009697,0.014826,-0.029489,0.750762,0.287941,-0.003682,0.003370,0.009697,0.014826,BENGALURU
1,0.0,0.0,0.0,0.002969,0.001421,0.005859,-0.029489,0.750762,0.287941,-0.003682,0.002969,0.001421,0.005859,BENGALURU(R)
2,0.0,0.0,0.0,0.035463,0.003575,0.000657,-0.029489,0.750762,0.287941,-0.003682,0.035463,0.003575,0.000657,RAMANAGARA
3,0.0,0.0,0.0,0.007854,0.001784,0.000000,-0.029489,0.750762,0.287941,-0.003682,0.007854,0.001784,0.000000,CHITRADURGA
4,0.0,0.0,0.0,0.852095,0.504480,0.269227,-0.029489,0.750762,0.287941,-0.003682,0.852095,0.504480,0.269227,DAVANAGERE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38594,1.0,1.0,1.0,0.033121,0.006570,0.000070,-0.029489,0.750762,0.287941,-0.003682,0.050000,0.050578,-3.441242,KALABURAGI
38595,1.0,1.0,1.0,0.426318,0.738523,0.734923,-0.029489,0.750762,0.287941,-0.003682,0.443196,0.782531,-2.706390,YADGIRI
38596,1.0,1.0,1.0,1.000000,1.000000,1.000000,-0.029489,0.750762,0.287941,-0.003682,1.016879,1.044008,-2.441312,RAICHUR
38597,1.0,1.0,1.0,0.438464,0.334119,0.096966,-0.029489,0.750762,0.287941,-0.003682,0.455342,0.378128,-3.344347,KOPPAL


In [292]:
result_df['New_Production'] = result_df['Beta_1'] * result_df['New_F1'] + result_df['Beta_2'] * result_df['New_F2'] + result_df['Beta_3'] * result_df['New_F3'] + result_df['Beta_0']
result_df

,Sensitivity_AreaUnderCereals_Paddy,Sensitivity_TotalNPK,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District,New_Production
0,0.0,0.0,0.0,0.003370,0.009697,0.014826,-0.029489,0.750762,0.287941,-0.003682,0.003370,0.009697,0.014826,BENGALURU,-0.024222
1,0.0,0.0,0.0,0.002969,0.001421,0.005859,-0.029489,0.750762,0.287941,-0.003682,0.002969,0.001421,0.005859,BENGALURU(R),-0.026873
2,0.0,0.0,0.0,0.035463,0.003575,0.000657,-0.029489,0.750762,0.287941,-0.003682,0.035463,0.003575,0.000657,RAMANAGARA,-0.001838
3,0.0,0.0,0.0,0.007854,0.001784,0.000000,-0.029489,0.750762,0.287941,-0.003682,0.007854,0.001784,0.000000,CHITRADURGA,-0.023079
4,0.0,0.0,0.0,0.852095,0.504480,0.269227,-0.029489,0.750762,0.287941,-0.003682,0.852095,0.504480,0.269227,DAVANAGERE,0.754500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38594,1.0,1.0,1.0,0.033121,0.006570,0.000070,-0.029489,0.750762,0.287941,-0.003682,0.050000,0.050578,-3.441242,KALABURAGI,0.035284
38595,1.0,1.0,1.0,0.426318,0.738523,0.734923,-0.029489,0.750762,0.287941,-0.003682,0.443196,0.782531,-2.706390,YADGIRI,0.538534
38596,1.0,1.0,1.0,1.000000,1.000000,1.000000,-0.029489,0.750762,0.287941,-0.003682,1.016879,1.044008,-2.441312,RAICHUR,1.043547
38597,1.0,1.0,1.0,0.438464,0.334119,0.096966,-0.029489,0.750762,0.287941,-0.003682,0.455342,0.378128,-3.344347,KOPPAL,0.433558


In [293]:
def normalize_list(lst):
    min_val = min(lst)
    max_val = max(lst)
    normalized_lst = [(x - min_val) / (max_val - min_val) for x in lst]
    return normalized_lst

In [294]:
import pandas as pd
import numpy as np

adjacency_excel = pd.read_excel('Karnataka_District_Adjacency_File.xlsx')

district_codes = sorted(set(adjacency_excel['District_Code']))

n = len(district_codes)
adj_matrix = np.zeros((n, n), dtype=int)

code_to_index = {code: idx for idx, code in enumerate(district_codes)}

for i in range(len(adjacency_excel)):
    district_code = adjacency_excel['District_Code'][i]
    neighbors = [int(num) for num in str(adjacency_excel['District_Neighbors'][i]).split(',') if num.strip().isdigit()]
    
    for neighbor in neighbors:
        adj_matrix[code_to_index[district_code]][code_to_index[neighbor]] = 1
        adj_matrix[code_to_index[neighbor]][code_to_index[district_code]] = 1  # Assuming adjacency is bidirectional

adj_matrix_df = pd.DataFrame(adj_matrix, index=district_codes, columns=district_codes)

print(adj_matrix_df)

adj_matrix_np = adj_matrix_df.to_numpy()

    1   2   3   4   5   6   7   8   9   10  ...  22  23  24  25  26  27  28  \
1    0   1   1   0   0   0   0   0   0   0  ...   0   0   0   0   0   0   0   
2    1   0   1   0   0   1   1   0   1   0  ...   0   0   0   0   0   0   0   
3    1   1   0   0   0   0   0   0   1   0  ...   0   0   0   0   0   0   0   
4    0   0   0   0   1   0   0   0   1   1  ...   0   0   0   1   0   0   0   
5    0   0   0   1   0   0   0   1   0   1  ...   0   1   0   1   0   0   0   
6    0   1   0   0   0   0   1   0   0   0  ...   0   0   0   0   0   0   0   
7    0   1   0   0   0   1   0   0   1   0  ...   0   0   0   0   0   0   0   
8    0   0   0   0   1   0   0   0   0   1  ...   0   1   1   0   0   0   0   
9    0   1   1   1   0   0   1   0   0   1  ...   0   0   0   0   0   0   0   
10   0   0   0   1   1   0   0   1   1   0  ...   0   0   0   0   0   0   0   
11   0   0   0   0   0   0   0   0   0   1  ...   0   0   0   0   0   0   0   
12   0   0   0   0   0   0   0   1   0   1  ...   0 

In [295]:
def calculate_avg_capability(capability, neighbors):
    num_neighbors = len(neighbors)
    if num_neighbors == 0:
        return 0
    sum_capabilities = sum(capability[n] for n in neighbors)
    return sum_capabilities / num_neighbors

def calculate_stress_stability(capability, adjacency_matrix):
    n = len(capability)
    stress = np.zeros(n)
    stability = np.zeros(n)
    
    for i in range(n):
        neighbors = [j for j in range(n) if adjacency_matrix[i][j] == 1 and i != j]
        if len(neighbors) > 0:
            avg_capability = calculate_avg_capability(capability, neighbors)
            stress[i] = abs(avg_capability - capability[i])
            stability[i] = 1 - stress[i]
        else:
            stress[i] = 0  # If no neighbors, stress is zero
            stability[i] = 1
        
    return stress, stability

In [296]:
result_df['Normalized_New_Prouction'] = normalize_list(result_df['New_Production'])
result_df

,Sensitivity_AreaUnderCereals_Paddy,Sensitivity_TotalNPK,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District,New_Production,Normalized_New_Prouction
0,0.0,0.0,0.0,0.003370,0.009697,0.014826,-0.029489,0.750762,0.287941,-0.003682,0.003370,0.009697,0.014826,BENGALURU,-0.024222,0.004920
1,0.0,0.0,0.0,0.002969,0.001421,0.005859,-0.029489,0.750762,0.287941,-0.003682,0.002969,0.001421,0.005859,BENGALURU(R),-0.026873,0.002450
2,0.0,0.0,0.0,0.035463,0.003575,0.000657,-0.029489,0.750762,0.287941,-0.003682,0.035463,0.003575,0.000657,RAMANAGARA,-0.001838,0.025780
3,0.0,0.0,0.0,0.007854,0.001784,0.000000,-0.029489,0.750762,0.287941,-0.003682,0.007854,0.001784,0.000000,CHITRADURGA,-0.023079,0.005985
4,0.0,0.0,0.0,0.852095,0.504480,0.269227,-0.029489,0.750762,0.287941,-0.003682,0.852095,0.504480,0.269227,DAVANAGERE,0.754500,0.730631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38594,1.0,1.0,1.0,0.033121,0.006570,0.000070,-0.029489,0.750762,0.287941,-0.003682,0.050000,0.050578,-3.441242,KALABURAGI,0.035284,0.060375
38595,1.0,1.0,1.0,0.426318,0.738523,0.734923,-0.029489,0.750762,0.287941,-0.003682,0.443196,0.782531,-2.706390,YADGIRI,0.538534,0.529366
38596,1.0,1.0,1.0,1.000000,1.000000,1.000000,-0.029489,0.750762,0.287941,-0.003682,1.016879,1.044008,-2.441312,RAICHUR,1.043547,1.000000
38597,1.0,1.0,1.0,0.438464,0.334119,0.096966,-0.029489,0.750762,0.287941,-0.003682,0.455342,0.378128,-3.344347,KOPPAL,0.433558,0.431536


In [297]:
stress_values_list = []
stability_values_list = []

# Define batch size
batch_size = case_file_length

# Iterate over the New_Production column in batches of 31
num_batches = len(result_df) // batch_size

for i in range(num_batches):
    # Get the batch of 31 production values
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch = normalize_list(result_df['New_Production'][start_idx:end_idx].values)
    
    # Calculate the stress and stability values for the batch
    stress_batch, stability_batch = calculate_stress_stability(batch, adj_matrix_np)
    
    stress_values_list.extend(stress_batch)
    stability_values_list.extend(stability_batch)


print(num_batches)
print(len(stress_batch))
print(len(stress_values_list), len(stability_values_list))

result_df['Stress_Values'] = stress_values_list
result_df['Stability_Values'] = stability_values_list

1331
29
38599 38599


In [298]:
result_df.columns

Index(['Sensitivity_AreaUnderCereals_Paddy', 'Sensitivity_TotalNPK',
       'Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated',
       'AreaUnderCereals_Paddy', 'TotalNPK',
       'NetAreaIrrigated_Canals_NetAreaIrrigated', 'Beta_0', 'Beta_1',
       'Beta_2', 'Beta_3', 'New_F1', 'New_F2', 'New_F3', 'District',
       'New_Production', 'Normalized_New_Prouction', 'Stress_Values',
       'Stability_Values'],
      dtype='object')

In [299]:
viz_df = result_df[['District','Sensitivity_AreaUnderCereals_Paddy','Sensitivity_TotalNPK','Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated',
                    'New_Production','Stability_Values','Normalized_New_Prouction']].copy()

abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
    'VIJAYANAGARA' : 'VJN'
}

viz_df['Abbreviation'] = viz_df['District'].map(abbreviation_mapping)

legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                    for abb, full_form in zip(viz_df['Abbreviation'].unique(), viz_df['District'].unique())]

In [300]:
viz_df['New_Production'] = viz_df['New_Production'].apply(lambda x: max(0, x))
y = y_st['Rice_Production']

viz_df['Percentage Change'] = viz_df['New_Production'] - np.tile(y.values, len(viz_df) // case_file_length + 1)[:len(viz_df)]
viz_df['Impact'] = viz_df['New_Production'] - np.tile(y.values, len(viz_df) // case_file_length + 1)[:len(viz_df)]

for i in range(0, len(viz_df), case_file_length):
    viz_df.loc[i:i+case_file_length-1, 'Percentage Change'] = viz_df.loc[i:i+case_file_length-1, 'Percentage Change'] * 100 / y.values + 1e-7
    viz_df.loc[i:i+case_file_length-1, 'Impact'] = viz_df.loc[i:i+case_file_length-1, 'Impact']
    # viz_df.loc[i:i+case_file_length-1, 'Normalized Percentage Change'] = normalize_list(viz_df.loc[i:i+case_file_length-1, 'Percentage Change'])

viz_df['Percentage Change'] = viz_df['Percentage Change'].replace([np.inf, -np.inf], 0)

# viz_df['Normalized Percentage Change'] = normalize_list(viz_df['Percentage Change'])

In [301]:
Working_Directory

,District,Rice_Production,TotalNPK,AreaUnderCereals_Paddy,NetAreaIrrigated_Canals_NetAreaIrrigated,Rice_Ratio
0,BENGALURU,3376.0,513.822990,782.0,9587.939256,0.022043
1,BENGALURU(R),2354.0,167.835355,736.0,9587.939256,0.008715
2,RAMANAGARA,12128.0,257.908893,4468.0,272.509463,0.034517
3,CHITRADURGA,4566.0,183.032957,1297.0,9.283245,0.005101
4,DAVANAGERE,489505.0,21197.529554,98259.0,18037.386429,0.212733
5,KOLAR,864.0,108.445953,395.0,9587.939256,0.004887
6,CHIKKABALLAPURA,2698.0,209.640893,1333.0,9587.939256,0.007393
7,SHIVAMOGGA,384974.0,17722.015760,109617.0,13305.677787,0.313060
8,TUMAKURU,26042.0,1069.562417,10379.0,114.924409,0.028037
9,CHIKKAMAGALURU,110533.0,14694.297645,33036.0,1344.644186,0.216529


In [311]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
import plotly.graph_objects as go

def update_plot(alpha1, alpha2, alpha3):
    
    tolerance = 1e-5 
    filtered_df = viz_df[
        (viz_df['Sensitivity_AreaUnderCereals_Paddy'].between(alpha1 - tolerance, alpha1 + tolerance)) &
        (viz_df['Sensitivity_TotalNPK'].between(alpha2 - tolerance, alpha2 + tolerance)) &
        (viz_df['Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated'].between(alpha3 - tolerance, alpha3 + tolerance)) 
    ]

    # Create a scatter plot with filtered data
    fig = px.scatter(filtered_df, x='Stability_Values', y='Percentage Change', title='Rice Impact vs Stability',
                     labels={'Percentage Change': 'Impact', 'Stability_Values': 'Stability'},
                     text='Abbreviation', opacity=1, color_discrete_sequence=['darkblue'])

    fig.update_traces(textposition='top center', textfont=dict(size=11))

    # Update layout to add annotations and legends
    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.25,
                showarrow=False,
                text=f'α<sub>AP</sub>: {alpha1:.2f}, α<sub>NPK</sub>: {alpha2:.2f}, α<sub>CN</sub>: {alpha3:.2f}',
                xref="paper",
                yref="paper",
                font=dict(size=13)
            )
        ],
        legend_title_text='Districts',
        legend_title=dict(font=dict(size=12)),
    )

    # Adding custom legends with abbreviations and full forms
    legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                     for abb, full_form in zip(filtered_df['Abbreviation'].unique(), filtered_df['District'].unique())]

    for label in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(color='darkblue'),
            name=f"{label['Abbreviation']}: {label['Full Form']}"
        ))

    # Adding average lines for stability and percentage change
    avg_stability = filtered_df['Stability_Values'].mean()
    avg_precent = filtered_df['Percentage Change'].mean()

    fig.add_shape(
        type='line',
        x0=avg_stability, y0=0, x1=avg_stability, y1=1,
        line=dict(color='red', width=1.5, dash='dash'),
        yref='paper'
    )

    fig.add_shape(
        type='line',
        x0=0, y0=avg_precent, x1=1, y1=avg_precent,
        line=dict(color='red', width=1.5, dash='dash'),
        xref='paper'
    )

    # Add annotations for average values
    fig.add_trace(go.Scatter(
        x=[avg_stability],
        y=[-0.07],
        text=[f'Avg Stability: {avg_stability:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    fig.add_trace(go.Scatter(
        x=[0.3],
        y=[avg_precent-0.04],
        text=[f'Avg Impact: {avg_precent:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    clear_output()

    display(fig)

# Creating sliders for interaction
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αAP:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αCN:')

# Creating interactive plot
interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider)

# Create update button to trigger plot update
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value))

# Display the button and sliders
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider])


interactive(children=(FloatSlider(value=0.5, description='αAP:', max=1.0), FloatSlider(value=0.5, description=…

In [303]:
Working_Directory[rows_to_remove]

,District,Rice_Production,TotalNPK,AreaUnderCereals_Paddy,NetAreaIrrigated_Canals_NetAreaIrrigated,Rice_Ratio
18,VIJAYAPURA,170.0,7.826782,51.0,12.617706,0.000144
19,BAGALKOT,85.0,7.747128,32.0,0.185064,0.000091


In [304]:
result_df.columns

Index(['Sensitivity_AreaUnderCereals_Paddy', 'Sensitivity_TotalNPK',
       'Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated',
       'AreaUnderCereals_Paddy', 'TotalNPK',
       'NetAreaIrrigated_Canals_NetAreaIrrigated', 'Beta_0', 'Beta_1',
       'Beta_2', 'Beta_3', 'New_F1', 'New_F2', 'New_F3', 'District',
       'New_Production', 'Normalized_New_Prouction', 'Stress_Values',
       'Stability_Values'],
      dtype='object')

In [305]:
# viz_df2 = result_df[['District','Sensitivity_SowingSeedsDistributed_Maize','Sensitivity_LivestockCensus_2012_Cattle','Sensitivity_AreaUnderCereals_Maize',
#                     'Sensitivity_TotalNPK',	'Sensitivity_NetAreaIrrigatedUnderDifferentSources_TubeWells_NetIrrigatedArea','New_F1','New_F2','New_F3','New_F4','New_F5']].copy()

viz_df2 = result_df[['District','Sensitivity_AreaUnderCereals_Paddy', 'Sensitivity_TotalNPK',
       'Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated','AreaUnderCereals_Paddy', 'TotalNPK',
       'NetAreaIrrigated_Canals_NetAreaIrrigated','New_F1','New_F2','New_F3']].copy()


viz_df2.columns

Index(['District', 'Sensitivity_AreaUnderCereals_Paddy',
       'Sensitivity_TotalNPK',
       'Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated',
       'AreaUnderCereals_Paddy', 'TotalNPK',
       'NetAreaIrrigated_Canals_NetAreaIrrigated', 'New_F1', 'New_F2',
       'New_F3'],
      dtype='object')

In [306]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
import plotly.graph_objects as go

def update_plot(alpha1, alpha2, alpha3):
    
    tolerance = 1e-5 
    filtered_df = viz_df[
        (viz_df['Sensitivity_AreaUnderCereals_Paddy'].between(alpha1 - tolerance, alpha1 + tolerance)) &
        (viz_df['Sensitivity_TotalNPK'].between(alpha2 - tolerance, alpha2 + tolerance)) &
        (viz_df['Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated'].between(alpha3 - tolerance, alpha3 + tolerance)) 
    ]

    # Create a scatter plot with filtered data
    fig = px.scatter(filtered_df, x='Stability_Values', y='Impact', title='Rice Impact vs Stability',
                     labels={'Impact': 'Impact', 'Stability_Values': 'Stability'},
                     text='Abbreviation', opacity=1, color_discrete_sequence=['darkblue'])

    fig.update_traces(textposition='top center', textfont=dict(size=11))

    # Update layout to add annotations and legends
    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.25,
                showarrow=False,
                text=f'α<sub>AP</sub>: {alpha1:.2f}, α<sub>NPK</sub>: {alpha2:.2f}, α<sub>CN</sub>: {alpha3:.2f}',
                xref="paper",
                yref="paper",
                font=dict(size=13)
            )
        ],
        legend_title_text='Districts',
        legend_title=dict(font=dict(size=12)),
    )

    # Adding custom legends with abbreviations and full forms
    legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                     for abb, full_form in zip(filtered_df['Abbreviation'].unique(), filtered_df['District'].unique())]

    for label in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(color='darkblue'),
            name=f"{label['Abbreviation']}: {label['Full Form']}"
        ))

    # Adding average lines for stability and percentage change
    avg_stability = filtered_df['Stability_Values'].mean()
    avg_precent = filtered_df['Impact'].mean()

    fig.add_shape(
        type='line',
        x0=avg_stability, y0=0, x1=avg_stability, y1=1,
        line=dict(color='red', width=1.5, dash='dash'),
        yref='paper'
    )

    fig.add_shape(
        type='line',
        x0=0, y0=avg_precent, x1=1, y1=avg_precent,
        line=dict(color='red', width=1.5, dash='dash'),
        xref='paper'
    )

    # Add annotations for average values
    fig.add_trace(go.Scatter(
        x=[avg_stability],
        y=[-0.07],
        text=[f'Avg Stability: {avg_stability:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    fig.add_trace(go.Scatter(
        x=[0.3],
        y=[avg_precent-0.04],
        text=[f'Avg Impact: {avg_precent:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    clear_output()

    display(fig)

# Creating sliders for interaction
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αAP:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αCN:')

# Creating interactive plot
interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider)

# Create update button to trigger plot update
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value))

# Display the button and sliders
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider])


interactive(children=(FloatSlider(value=0.5, description='αAP:', max=1.0), FloatSlider(value=0.5, description=…

In [307]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output

# Function to update the bar plot based on sensitivity sliders
def update_bar_plot(alpha1, alpha2, alpha3):
    tolerance = 1e-5
    
    # Filter the dataframe based on slider sensitivity values
    filtered_df = viz_df2[
        (viz_df2['Sensitivity_AreaUnderCereals_Paddy'].between(alpha1 - tolerance, alpha1 + tolerance)) &
        (viz_df2['Sensitivity_TotalNPK'].between(alpha2 - tolerance, alpha2 + tolerance)) &
        (viz_df2['Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated'].between(alpha3 - tolerance, alpha3 + tolerance)) 
    ]
    
    # Define the factors and their display names
    factors = ['New_F1', 'New_F2', 'New_F3']
    display_names = ['Area_Paddy(Ha)', 'TotalNPK(Tonnes)', 'Area_Canals(Ha)']

    # Calculate mean values for the selected sensitivity group
    user_values = filtered_df[factors].mean()
    
    # Get the mean values for the original case where all sensitivities are zero
    original_values = viz_df2[viz_df2[['Sensitivity_AreaUnderCereals_Paddy',
                                       'Sensitivity_TotalNPK', 'Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated']].eq(0).all(axis=1)][factors].mean()
    
    user_values_formatted = [f'{val:.2f}' for val in user_values]
    original_values_formatted = [f'{val:.2f}' for val in original_values]
    
    # Create the bar plot with interchanged axes, adding text labels
    fig = go.Figure(data=[
        go.Bar(
            name='Original Values (Sensitivity=0)', 
            y=display_names, 
            x=original_values, 
            marker_color='blue', 
            orientation='h',
            text=original_values_formatted,  # Add labels with the values of the bars
            textposition='auto'    # Automatically position the text inside or outside the bar
        ),
        go.Bar(
            name=f'Chosen Sensitivity [{alpha1}, {alpha2}, {alpha3}]', 
            y=display_names, 
            x=user_values, 
            marker_color='orange', 
            orientation='h',
            text=user_values_formatted,  # Add labels with the values of the bars
            textposition='auto'    # Automatically position the text inside or outside the bar
        )
    ])
    
    # Customize the layout
    fig.update_layout(
        title="Comparison of Factor Values for Original and Chosen Sensitivity",
        yaxis_title="Factors",
        xaxis_title="Values",
        barmode='group'
    )

    # Clear previous output and display updated plot
    clear_output()
    display(fig)


# Creating sliders for interaction
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αAP:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αCN:')

# Creating interactive plot
interactive_plot = interact(update_bar_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider)

# Create update button to trigger plot update
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_bar_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value))

# Display the button and sliders
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider])

interactive(children=(FloatSlider(value=0.5, description='αAP:', max=1.0), FloatSlider(value=0.5, description=…

In [308]:
result_df['New_Production'].mean()

0.3024051344915847

In [309]:
result_df

,Sensitivity_AreaUnderCereals_Paddy,Sensitivity_TotalNPK,Sensitivity_NetAreaIrrigated_Canals_NetAreaIrrigated,AreaUnderCereals_Paddy,TotalNPK,NetAreaIrrigated_Canals_NetAreaIrrigated,Beta_0,Beta_1,Beta_2,Beta_3,New_F1,New_F2,New_F3,District,New_Production,Normalized_New_Prouction,Stress_Values,Stability_Values
0,0.0,0.0,0.0,0.003370,0.009697,0.014826,-0.029489,0.750762,0.287941,-0.003682,0.003370,0.009697,0.014826,BENGALURU,-0.024222,0.004920,0.009533,0.990467
1,0.0,0.0,0.0,0.002969,0.001421,0.005859,-0.029489,0.750762,0.287941,-0.003682,0.002969,0.001421,0.005859,BENGALURU(R),-0.026873,0.002450,0.019037,0.980963
2,0.0,0.0,0.0,0.035463,0.003575,0.000657,-0.029489,0.750762,0.287941,-0.003682,0.035463,0.003575,0.000657,RAMANAGARA,-0.001838,0.025780,0.105923,0.894077
3,0.0,0.0,0.0,0.007854,0.001784,0.000000,-0.029489,0.750762,0.287941,-0.003682,0.007854,0.001784,0.000000,CHITRADURGA,-0.023079,0.005985,0.282781,0.717219
4,0.0,0.0,0.0,0.852095,0.504480,0.269227,-0.029489,0.750762,0.287941,-0.003682,0.852095,0.504480,0.269227,DAVANAGERE,0.754500,0.730631,0.448834,0.551166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38594,1.0,1.0,1.0,0.033121,0.006570,0.000070,-0.029489,0.750762,0.287941,-0.003682,0.050000,0.050578,-3.441242,KALABURAGI,0.035284,0.060375,0.379028,0.620972
38595,1.0,1.0,1.0,0.426318,0.738523,0.734923,-0.029489,0.750762,0.287941,-0.003682,0.443196,0.782531,-2.706390,YADGIRI,0.538534,0.529366,0.487920,0.512080
38596,1.0,1.0,1.0,1.000000,1.000000,1.000000,-0.029489,0.750762,0.287941,-0.003682,1.016879,1.044008,-2.441312,RAICHUR,1.043547,1.000000,0.637798,0.362202
38597,1.0,1.0,1.0,0.438464,0.334119,0.096966,-0.029489,0.750762,0.287941,-0.003682,0.455342,0.378128,-3.344347,KOPPAL,0.433558,0.431536,0.111283,0.888717


In [310]:
result_df.to_csv('Rice_Prescriptive_5000.csv',index=False)